In [1]:
from google.cloud import bigquery
import pandas as pd
import google
import os

# Establece la variable de entorno GOOGLE_APPLICATION_CREDENTIALS con la ruta al archivo de credenciales
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/raul.raggio/AppData/Roaming/gcloud/application_default_credentials.json"
credentials, project = google.auth.default()

project="peya-chile"


client = bigquery.Client(project=project, credentials=credentials)
print("Client creating using default project: {}".format(client.project))

Client creating using default project: peya-chile


c:\Users\raul.raggio\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [2]:
flag=1

In [3]:

if flag ==0:
    
    sql1 = """

       


    SELECT
    logistic_orders.country.country_name  AS logistic_orders_country__country_name,
        (FORMAT_DATE('%F', DATE_TRUNC(logistic_orders.created_date_local , WEEK(MONDAY)))) AS logistic_orders_created_date_local_week,
(logistic_orders.created_date_local ) AS logistic_orders_created_date_local_date,
        
    dim_partner.partner_name  AS dim_partner_partner_name,
    dim_partner.partner_id  AS dim_partner_partner_id,
logistic_orders.peya_order_id,
logistic_orders.order_status,
    
    (FORMAT_DATE('%A', logistic_orders.created_date_local )) AS logistic_orders_created_date_local_day_of_week,
    (EXTRACT(HOUR FROM logistic_orders.created_at_local )) AS logistic_orders_created_at_local_hour_of_day,
    (EXTRACT(MONTH FROM logistic_orders.created_at_local )) AS logistic_orders_created_at_local_month,
    dim_partner.main_cousine_category_name as cuisine,
    dim_partner.franchise.franchise_name as franchise_name,
    dim_partner.business_type.business_type_name,
    logistic_orders.city.city_name as city,
     fact_logistic_orders__zone.zone_name  AS zone,



logistic_orders.is_vendor_late_10 as is_vendor_late_10,
logistic_orders.is_vendor_late_nn as is_vendor_late_nn,

 logistic_orders.is_rider_late_10 as is_rider_late_10,

logistic_orders.is_rider_late_nn as is_rider_late_nn,




logistic_orders.estimated_prep_time/60 as estimated_prep_time,


CASE WHEN ( logistic_orders.timings.actual_delivery_time*1.000/60  ) > 45 THEN 1 ELSE 0 END
  AS slow_delivery,

CASE WHEN ( fact_orders.promisedDeliveryTime.maxMinutes   + 10) < ( logistic_orders.timings.actual_delivery_time*1.000/60  )  THEN 1  ELSE 0 END AS lcf,


 
logistic_orders.timings.at_vendor_time_cleaned/60 at_vendor_time_cleaned,
logistic_orders.timings.at_vendor_time/60 at_vendor_time,
logistic_orders.timings.assumed_actual_preparation_time/60 as assumed_prep_time,
logistic_orders.timings.promised_delivery_time/60 as promised_delivery_time,

logistic_orders.timings.actual_delivery_time/60 as actual_delivery_time,
 

logistic_orders.timings.vendor_late/60 as vendor_late_min,
logistic_orders.timings.rider_late/60 as rider_late_min,
logistic_orders.timings.hold_back_time/60 as hold_back_time_min,
logistic_orders.timings.estimated_walk_in_duration/60 as estimated_walk_in_min,
logistic_orders.timings.estimated_walk_out_duration/60 as estimated_walk_out_min,

logistic_orders.timings.to_customer_time/60 as to_customer_time_min,
  
ROUND(DATETIME_DIFF(rider.food_is_ready_at,rider.sent_to_vendor_at, SECOND)/60,1) AS RPT,
CASE WHEN rider.food_is_ready_at IS NOT NULL THEN 1 ELSE 0 END AS orders_with_fir,
  ROUND(rider.estimated_prep_time/60,1) AS EPT,
  ROUND(rider.estimated_prep_buffer/60,1) AS buffer,


FROM `peya-bi-tools-pro.il_logistics.fact_logistic_orders` AS logistic_orders
LEFT JOIN `peya-bi-tools-pro.il_core.fact_orders`  AS fact_orders ON fact_orders.order_id= logistic_orders.peya_order_id and fact_orders.registered_date = logistic_orders.created_date_local

LEFT JOIN `peya-bi-tools-pro.il_core.dim_partner` AS dim_partner ON dim_partner.partner_id=fact_orders.restaurant.id
LEFT JOIN UNNEST (logistic_orders.deliveries) as fact_logistic_orders__deliveries
LEFT JOIN UNNEST([logistic_orders.zone]) as fact_logistic_orders__zone

LEFT JOIN  `peya-data-origins-pro.cl_hurrier.orders_v2` ov2 on SAFE_CAST(ov2.global_order_id AS INT)=logistic_orders.peya_order_id


  WHERE ((logistic_orders.created_date_local) between  DATE_SUB(current_date(), INTERVAL 3 WEEK)  and current_date() )

AND UPPER( logistic_orders.country.country_name  ) = UPPER('chile')
AND (NOT (logistic_orders.is_preorder ) OR (logistic_orders.is_preorder ) IS NULL)


 AND ((fact_orders.restaurant.id ) > 0 OR (fact_orders.restaurant.id ) IS NULL) 
 AND (logistic_orders.peya_order_id ) IS NOT NULL



ORDER BY 15 DESC

        """
    data1=client.query(sql1).to_dataframe()
    data1.to_csv("data1.csv")
else:
    
    data1=pd.read_csv("data1.csv")

In [4]:
data1

,Unnamed: 0,logistic_orders_country__country_name,logistic_orders_created_date_local_week,logistic_orders_created_date_local_date,dim_partner_partner_name,dim_partner_partner_id,peya_order_id,order_status,logistic_orders_created_date_local_day_of_week,logistic_orders_created_at_local_hour_of_day,...,vendor_late_min,rider_late_min,hold_back_time_min,estimated_walk_in_min,estimated_walk_out_min,to_customer_time_min,RPT,orders_with_fir,EPT,buffer
0,0,Chile,2025-03-03,2025-03-05,McDonald's Alto Las Condes,67499.0,1474605336,completed,Wednesday,13,...,1.300000,-3.466667,3.983333,5.000,5.000,7.900000,3.0,1,13.0,2.0
1,1,Chile,2025-03-03,2025-03-04,Chicken Love You - Geronimo De Alderete,411940.0,1473803619,completed,Tuesday,17,...,NaN,10.733333,1.000000,1.050,2.233,3.866667,NaN,0,14.0,0.0
2,2,Chile,2025-02-10,2025-02-13,Lulupok,131026.0,1450963875,completed,Thursday,12,...,10.316667,2.933333,0.383333,8.000,8.000,15.250000,NaN,0,15.0,1.0
3,3,Chile,2025-02-24,2025-02-26,Kokai Sushi - Vitacura,415060.0,1466380426,completed,Wednesday,19,...,0.783333,7.100000,1.116667,2.100,1.650,5.533333,23.9,1,37.0,2.0
4,4,Chile,2025-03-03,2025-03-05,Margó - Alto Las Condes,526131.0,1474899300,completed,Wednesday,17,...,16.266667,4.433333,1.150000,5.183,4.683,20.816667,20.9,1,17.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1858311,1858311,Chile,2025-02-24,2025-02-26,Churrascon 24/7,259095.0,1466026982,cancelled,Wednesday,13,...,NaN,11.066667,11.300000,0.967,2.450,6.316667,NaN,0,10.0,0.0
1858312,1858312,Chile,2025-02-24,2025-02-26,Churrascon 24/7,259095.0,1466078700,completed,Wednesday,14,...,0.966667,1.950000,0.183333,0.967,2.450,9.533333,NaN,0,10.0,0.0
1858313,1858313,Chile,2025-02-24,2025-03-01,Churrascon 24/7,259095.0,1470767630,cancelled,Saturday,23,...,NaN,1.233333,17.500000,0.967,2.450,0.000000,NaN,0,10.0,0.0
1858314,1858314,Chile,2025-03-03,2025-03-05,Churrascon 24/7,259095.0,1475389455,cancelled,Wednesday,23,...,0.333333,7.900000,0.716667,0.967,2.450,0.000000,NaN,0,11.0,0.0


In [5]:
df1=data1

In [6]:
df1.columns

Index(['Unnamed: 0', 'logistic_orders_country__country_name',
       'logistic_orders_created_date_local_week',
       'logistic_orders_created_date_local_date', 'dim_partner_partner_name',
       'dim_partner_partner_id', 'peya_order_id', 'order_status',
       'logistic_orders_created_date_local_day_of_week',
       'logistic_orders_created_at_local_hour_of_day',
       'logistic_orders_created_at_local_month', 'cuisine', 'franchise_name',
       'business_type_name', 'city', 'zone', 'is_vendor_late_10',
       'is_vendor_late_nn', 'is_rider_late_10', 'is_rider_late_nn',
       'estimated_prep_time', 'slow_delivery', 'lcf', 'at_vendor_time_cleaned',
       'at_vendor_time', 'assumed_prep_time', 'promised_delivery_time',
       'actual_delivery_time', 'vendor_late_min', 'rider_late_min',
       'hold_back_time_min', 'estimated_walk_in_min', 'estimated_walk_out_min',
       'to_customer_time_min', 'RPT', 'orders_with_fir', 'EPT', 'buffer'],
      dtype='object')

In [7]:
df1["logistic_orders_created_date_local_week"].min()

'2025-02-10'

In [8]:
    df1=data1
    df1['logistic_orders_created_date_local_date'] = pd.to_datetime(df1['logistic_orders_created_date_local_date'])

    df1['franchise_name'] = df1['franchise_name'].str.lower()
    df1['cuisine'].fillna("no-cuisine", inplace=True)
    df1['city'].fillna("no city", inplace=True)
    df1['zone'].fillna("no zone", inplace=True)
    df1['franchise_name'].fillna("no franchise", inplace=True)
    df1['business_type_name'].fillna("no business", inplace=True)
    df1["month"] = df1["logistic_orders_created_date_local_date"].dt.month
    df1['logistic_orders_created_date_local_week'] = pd.to_datetime(df1['logistic_orders_created_date_local_week'])
    df1['ept_atvc'] = df1['estimated_prep_time'] + df1['at_vendor_time_cleaned']


C:\Users\raul.raggio\AppData\Local\Temp\ipykernel_20688\3743998716.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df1['cuisine'].fillna("no-cuisine", inplace=True)
C:\Users\raul.raggio\AppData\Local\Temp\ipykernel_20688\3743998716.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

In [9]:
df1["collection_time"] = df1["at_vendor_time"] - df1["estimated_walk_in_min"]
df1["rider_arrival"] = df1["rider_late_min"] + df1["estimated_walk_in_min"]
df1 = df1.dropna(subset=['rider_arrival', 'collection_time'])

In [10]:
#partners cambios performance
dfaux=df1[df1["business_type_name"]=="Restaurant"]
info_partner=dfaux.groupby(["dim_partner_partner_id","dim_partner_partner_name"]).agg({"peya_order_id":"nunique","is_vendor_late_10":"sum","is_vendor_late_nn":"sum","is_rider_late_10":"sum","is_rider_late_nn":"sum",'estimated_prep_time':'mean','at_vendor_time_cleaned':'mean'}).reset_index()
info_partner["%vendor_late"]=info_partner["is_vendor_late_10"]/info_partner["is_vendor_late_nn"]
info_partner["%rider_late"]=info_partner["is_rider_late_10"]/info_partner["is_rider_late_nn"]
cambiostotal1=info_partner[(info_partner["%vendor_late"]<=0.08) & (info_partner["peya_order_id"]>=90) ]
cambiostotal1["new_ept"]=round(cambiostotal1["estimated_prep_time"]-(6-cambiostotal1["at_vendor_time_cleaned"]))
cambiostotal1.loc[cambiostotal1["new_ept"] < 5, "new_ept"] = 5
cambiostotal1

C:\Users\raul.raggio\AppData\Local\Temp\ipykernel_20688\4179524387.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cambiostotal1["new_ept"]=round(cambiostotal1["estimated_prep_time"]-(6-cambiostotal1["at_vendor_time_cleaned"]))


,dim_partner_partner_id,dim_partner_partner_name,peya_order_id,is_vendor_late_10,is_vendor_late_nn,is_rider_late_10,is_rider_late_nn,estimated_prep_time,at_vendor_time_cleaned,%vendor_late,%rider_late,new_ept
1,828.0,Pita House,169,5,164,5,169,11.639053,2.115644,0.030488,0.029586,8.0
12,5182.0,Il Romano,168,3,143,21,164,15.970414,1.988356,0.020979,0.128049,12.0
14,5372.0,Jhot Pizza El Bosque,117,5,96,17,113,15.034188,2.115646,0.052083,0.150442,11.0
17,5375.0,Jhot Pizza San Bernardo,273,16,251,19,270,14.754579,2.649471,0.063745,0.070370,11.0
23,6212.0,Fuente Suiza Ñuñoa,1229,88,1180,43,1223,15.426829,2.457016,0.074576,0.035159,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...
11426,537321.0,Grido Avenida Argentina,119,1,107,9,116,7.042017,1.406019,0.009346,0.077586,5.0
11436,537545.0,Umai Sushi - Temuco,286,9,272,14,286,13.797203,2.159681,0.033088,0.048951,10.0
11440,537639.0,Chucao Burger Sandwich,197,13,189,14,203,15.852941,2.528070,0.068783,0.068966,12.0
11455,538038.0,Mao Sushi - La Florida,183,12,175,7,182,14.688525,1.777462,0.068571,0.038462,10.0


In [11]:

#ver histogramas
cambiostotal2=info_partner[(info_partner["%vendor_late"]>0.08) & (info_partner["%vendor_late"]<=0.15) & (info_partner["peya_order_id"]>=90) ]
idpartners=cambiostotal2["dim_partner_partner_id"].values
idpartners
solucion_eje_x=5

limite_rider_arrival=10
limite_collection_time=25


In [12]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.optimize import curve_fit
# Inicializar una lista para guardar los resultados combinados
results_combined = []

# Iterar sobre cada id en el array idspartnerssospechosos
for partner_id_estudio in idpartners:
    dfchart = df1[
        (df1["dim_partner_partner_id"] == partner_id_estudio) &
        (df1["rider_arrival"] <= limite_rider_arrival) &
        (df1["rider_arrival"] >= -limite_rider_arrival) &  
        (df1["collection_time"] >= -limite_collection_time) & 
        (df1["collection_time"] <= limite_collection_time)
    ]
    #print(partner_id_estudio)

    if len(dfchart) == 0:
        continue

    partner_name = dfchart["dim_partner_partner_name"].iloc[0]

    # Definir la función lineal por tramos
    def piecewise_linear(x, x0, y0, k1, k2):
        return np.where(x < x0, k1 * (x - x0) + y0, k2 * (x - x0) + y0)

    bounds = ([-solucion_eje_x, -np.inf, -np.inf, -np.inf], [solucion_eje_x, np.inf, np.inf, np.inf])
    
    # Ajustar el modelo a los datos "despues"
    p0 = [0, 0, -1, 0]
    popt, _ = curve_fit(piecewise_linear, dfchart['rider_arrival'], dfchart['collection_time'], p0, bounds=bounds, maxfev=10000)

    x_offset = popt[0]
    y_offset = popt[1]
    
    # Calcular el porcentaje de puntos después del cambio de pendiente
    total_points = len(dfchart)
    points_after = len(dfchart[dfchart['rider_arrival'] > x_offset])
    points_after_abajo = len(dfchart[(dfchart['rider_arrival'] > x_offset) & (dfchart['collection_time'] <= y_offset)])
    percentage_after = (points_after / total_points) * 100

    # Guardar los parámetros ajustados
    results_combined.append({
        'partner_id': partner_id_estudio,
        'partner_name': partner_name,
        'x0': popt[0],
        'y0': popt[1],
        'k1': popt[2],
        'k2': popt[3],
        '%_despues': percentage_after,
    })

results_df = pd.DataFrame(results_combined)

In [15]:
df1.columns

Index(['Unnamed: 0', 'logistic_orders_country__country_name',
       'logistic_orders_created_date_local_week',
       'logistic_orders_created_date_local_date', 'dim_partner_partner_name',
       'dim_partner_partner_id', 'peya_order_id', 'order_status',
       'logistic_orders_created_date_local_day_of_week',
       'logistic_orders_created_at_local_hour_of_day',
       'logistic_orders_created_at_local_month', 'cuisine', 'franchise_name',
       'business_type_name', 'city', 'zone', 'is_vendor_late_10',
       'is_vendor_late_nn', 'is_rider_late_10', 'is_rider_late_nn',
       'estimated_prep_time', 'slow_delivery', 'lcf', 'at_vendor_time_cleaned',
       'at_vendor_time', 'assumed_prep_time', 'promised_delivery_time',
       'actual_delivery_time', 'vendor_late_min', 'rider_late_min',
       'hold_back_time_min', 'estimated_walk_in_min', 'estimated_walk_out_min',
       'to_customer_time_min', 'RPT', 'orders_with_fir', 'EPT', 'buffer',
       'month', 'ept_atvc', 'collection_time',

In [19]:
results_preliminar=results_df[(results_df["x0"]<=-3) & (results_df["k1"]<-1) & (results_df["k2"]<=0)  & (results_df["k2"]>=results_df["k1"])  & (results_df["%_despues"]<=80)   & (results_df["%_despues"]>=20)  ] 
info_partner=df1[(df1["dim_partner_partner_id"].isin(results_preliminar["partner_id"].values))].groupby(["dim_partner_partner_id","dim_partner_partner_name"]).agg({"peya_order_id":"nunique","is_vendor_late_10":"sum","is_vendor_late_nn":"sum","is_rider_late_10":"sum","is_rider_late_nn":"sum","estimated_prep_time":"mean"}).reset_index()
info_partner["%vendor_late"]=info_partner["is_vendor_late_10"]/info_partner["is_vendor_late_nn"]
info_partner["%rider_late"]=info_partner["is_rider_late_10"]/info_partner["is_rider_late_nn"]
info_partner_total=info_partner.merge(results_preliminar,how="left",left_on="dim_partner_partner_id",right_on="partner_id")
info_partner_total=info_partner_total.sort_values("peya_order_id",ascending=False)
cambiostotal2=info_partner_total
cambiostotal2["new_ept"]=round(cambiostotal2["estimated_prep_time"]+cambiostotal2["x0"])
cambiostotal2.loc[cambiostotal2["new_ept"] < 5, "new_ept"] = 5
cambiostotal2

,dim_partner_partner_id,dim_partner_partner_name,peya_order_id,is_vendor_late_10,is_vendor_late_nn,is_rider_late_10,is_rider_late_nn,estimated_prep_time,%vendor_late,%rider_late,partner_id,partner_name,x0,y0,k1,k2,%_despues,new_ept
24,477691.0,Mcdonald's Talca Colin,975,100,965,8,973,9.833846,0.103627,0.008222,477691.0,Mcdonald's Talca Colin,-4.863467,5.839113,-1.706992,-0.263269,67.573222,5.0
14,263727.0,Papa John's Pizza - Calama Sur,966,104,942,15,957,27.038302,0.110403,0.015674,263727.0,Papa John's Pizza - Calama Sur,-4.783337,4.350095,-1.300868,-0.110119,76.865672,22.0
5,107235.0,Burger King® - Terval Valdivia,937,91,910,21,931,11.716720,0.100000,0.022556,107235.0,Burger King® - Terval Valdivia,-5.000000,4.730710,-1.098012,-0.086911,63.474388,7.0
20,407767.0,Full Burger,834,88,820,8,828,16.657074,0.107317,0.009662,407767.0,Full Burger,-4.466901,5.089909,-1.986999,-0.231977,50.061200,12.0
19,335604.0,Burger King® - Mac Iver,557,45,523,27,550,10.297491,0.086042,0.049091,335604.0,Burger King® - Mac Iver,-3.000018,3.703580,-1.139790,-0.211117,69.066148,7.0
7,133607.0,Papa John's Pizza - Puente Alto,527,67,481,40,521,18.091082,0.139293,0.076775,133607.0,Papa John's Pizza - Puente Alto,-3.216316,3.682126,-1.052713,-0.177911,55.991285,15.0
13,253656.0,Tsuki Sushi !,496,42,430,64,494,10.798387,0.097674,0.129555,253656.0,Tsuki Sushi !,-3.528185,4.638210,-1.858317,-0.283853,79.058824,7.0
18,334344.0,Don Wrap,492,68,487,4,491,16.815041,0.139630,0.008147,334344.0,Don Wrap,-5.000000,5.376534,-1.207660,-0.323657,75.467775,12.0
10,166477.0,Johnny Rockets - Puente Alto,456,39,423,31,454,20.122807,0.092199,0.068282,166477.0,Johnny Rockets - Puente Alto,-3.350329,2.940994,-1.247078,-0.301206,69.801980,17.0
1,61361.0,La Pamplona,403,48,387,12,399,17.305211,0.124031,0.030075,61361.0,La Pamplona,-3.115475,2.845067,-1.401226,-0.146695,48.684211,14.0


In [20]:


df1["diff_fir"]=df1["RPT"]-df1["EPT"]
df1["Ratio"]=df1["RPT"]/df1["EPT"]
df1["ci"]=df1["EPT"]-2*df1["buffer"]
df1["fir_considerado"] = df1.apply(lambda row: 1 if row["RPT"] > row["ci"] else 0, axis=1)
dfaux=df1[(df1["business_type_name"].isin(["Restaurant","Coffee"]))  ]
grouped = dfaux.groupby(['dim_partner_partner_id','dim_partner_partner_name']).agg({'EPT':'mean','RPT': 'mean','Ratio':'mean','orders_with_fir':'sum','peya_order_id':'nunique','diff_fir':'mean','fir_considerado':'sum','is_vendor_late_10':'sum','is_vendor_late_nn':'sum'})
grouped["%vendorlate"]=grouped["is_vendor_late_10"]/grouped["is_vendor_late_nn"]
grouped["%fir"]=grouped["orders_with_fir"]/grouped["peya_order_id"]
grouped["%fir_Considerado"]=grouped["fir_considerado"]/grouped["orders_with_fir"]
prueba=pd.DataFrame(grouped[["EPT","RPT","Ratio","diff_fir","orders_with_fir","peya_order_id","%fir","%fir_Considerado",'%vendorlate']])

prueba=prueba.reset_index()
dffir=prueba.copy()
dffir

dfaux=df1[ (df1["orders_with_fir"]==1) & (df1["Ratio"]<=0.1) & (df1["business_type_name"].isin(["Restaurant","Coffee"])) ]
grouped = dfaux.groupby(['dim_partner_partner_id']).agg({'RPT':'mean','orders_with_fir':'sum'})


prueba=pd.DataFrame(grouped)
prueba=prueba.reset_index()
partnersmalfir=prueba.copy()
partnersmalfir.columns=["dim_partner_partner_id","RPT_bajo","orders_with_fir_bajo"]

dffir=dffir.merge(partnersmalfir, on="dim_partner_partner_id", how="left")
dffir["%_fir_bajo"]=dffir["orders_with_fir_bajo"]/dffir["orders_with_fir"]
dffir["%_fir_bajo"].fillna(0, inplace=True)
dffir

C:\Users\raul.raggio\AppData\Local\Temp\ipykernel_20688\1838356809.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["diff_fir"]=df1["RPT"]-df1["EPT"]
C:\Users\raul.raggio\AppData\Local\Temp\ipykernel_20688\1838356809.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["Ratio"]=df1["RPT"]/df1["EPT"]
C:\Users\raul.raggio\AppData\Local\Temp\ipykernel_20688\1838356809.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

,dim_partner_partner_id,dim_partner_partner_name,EPT,RPT,Ratio,diff_fir,orders_with_fir,peya_order_id,%fir,%fir_Considerado,%vendorlate,RPT_bajo,orders_with_fir_bajo,%_fir_bajo
0,77.0,Subway Parque Arauco - 41767,10.114286,12.088889,1.217912,2.088889,9,35,0.257143,0.333333,0.074074,NaN,NaN,0.000000
1,828.0,Pita House,11.639053,11.304808,0.985707,-0.156731,104,169,0.615385,0.634615,0.030488,NaN,NaN,0.000000
2,849.0,Pasta Pazza,22.000000,23.900000,0.885185,-3.100000,1,7,0.142857,1.000000,0.166667,NaN,NaN,0.000000
3,1031.0,Niu Sushi - Tobalaba,44.791480,35.312923,0.790587,-9.431692,325,446,0.728700,0.273846,0.136020,NaN,NaN,0.000000
4,1869.0,Tarragona Independencia Rancagua,12.662379,9.689109,0.753479,-3.068317,202,311,0.649518,0.405941,0.135135,0.918182,22.0,0.108911
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11598,541035.0,Serendipia Burgers Chiguayante,13.000000,10.900000,0.838462,-2.100000,1,1,1.000000,0.000000,0.000000,NaN,NaN,0.000000
11599,541047.0,Matsumoto Nikkei Independencia,13.000000,NaN,NaN,NaN,0,3,0.000000,NaN,0.000000,NaN,NaN,0.000000
11600,541123.0,Laldea,13.000000,24.100000,1.853846,11.100000,1,1,1.000000,1.000000,NaN,NaN,NaN,0.000000
11601,541146.0,Delicias Del Bajón,13.000000,9.900000,0.761538,-3.100000,1,1,1.000000,0.000000,0.000000,NaN,NaN,0.000000


In [22]:
def no_cambio(row):
 
   
    if (row["%fir"] >=0.6)   &  (row["EPT"] >=10)   &  (row["Ratio"] <0.6)    &  (row["diff_fir"] <=-3)   &  (row["peya_order_id"] >90) &  (row["%vendorlate"] <0.20)    :
    
        return 1
   
    
    else:
        return 0



dffir['fir_data_baja'] = dffir.apply(no_cambio,axis=1)
cambiostotal3=dffir[(dffir["fir_data_baja"]==1) & (~dffir["dim_partner_partner_id"].isin(cambiostotal2["dim_partner_partner_id"].values)) & (~dffir["dim_partner_partner_id"].isin(cambiostotal1["dim_partner_partner_id"].values))]
cambiostotal3["new_ept"]=round(cambiostotal3["RPT"]+1)
cambiostotal3

C:\Users\raul.raggio\AppData\Local\Temp\ipykernel_20688\2263664958.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cambiostotal3["new_ept"]=round(cambiostotal3["RPT"]+1)


,dim_partner_partner_id,dim_partner_partner_name,EPT,RPT,Ratio,diff_fir,orders_with_fir,peya_order_id,%fir,%fir_Considerado,%vendorlate,RPT_bajo,orders_with_fir_bajo,%_fir_bajo,fir_data_baja,new_ept
295,52576.0,Tarragona Florida Center,10.421687,5.612195,0.558805,-4.721138,123,166,0.740964,0.300813,0.087500,0.800000,6.0,0.048780,1,7.0
421,58460.0,McDonald's Santa Rosa,20.873842,7.361242,0.359286,-12.897859,934,1395,0.669534,0.125268,0.153005,1.252657,207.0,0.221627,1,8.0
630,62969.0,Tarragona Terminal Rancagua,13.527778,4.500258,0.338574,-9.015206,388,431,0.900232,0.128866,0.183824,0.885859,99.0,0.255155,1,6.0
632,62973.0,Tarragona - Rancagua Las Américas,12.910150,7.006011,0.540576,-5.947541,366,601,0.608985,0.341530,0.133913,0.820290,69.0,0.188525,1,8.0
957,67826.0,Taco Bell Mall Plaza Tobalaba,12.872340,5.848889,0.455608,-7.106667,90,94,0.957447,0.022222,0.177215,NaN,NaN,0.000000,1,7.0
1333,104501.0,McDonald's Macul,13.542833,3.853881,0.293574,-9.702689,1971,2306,0.854727,0.133435,0.192999,0.880157,635.0,0.322171,1,5.0
1576,110772.0,Tarragona Lider El Belloto,16.820375,5.442105,0.323074,-11.495029,684,745,0.918121,0.109649,0.162311,1.038743,191.0,0.279240,1,6.0
1738,116336.0,Pizza Hut - Mall Florida Center,21.537037,5.326415,0.252088,-16.192453,106,108,0.981481,0.000000,0.177083,NaN,NaN,0.000000,1,6.0
2217,133522.0,Papa John's Pizza - Mall Plaza Trebol,13.979058,6.532065,0.468255,-7.489674,184,191,0.963351,0.195652,0.096591,0.857895,19.0,0.103261,1,8.0
2250,133683.0,Papa John's Pizza - Viña Centro,17.800464,5.888525,0.333390,-11.924122,427,431,0.990719,0.032787,0.129380,1.234884,43.0,0.100703,1,7.0


In [23]:
cambios_totales=pd.concat([cambiostotal1,cambiostotal2,cambiostotal3])
cambios_totales

,dim_partner_partner_id,dim_partner_partner_name,peya_order_id,is_vendor_late_10,is_vendor_late_nn,is_rider_late_10,is_rider_late_nn,estimated_prep_time,at_vendor_time_cleaned,%vendor_late,...,Ratio,diff_fir,orders_with_fir,%fir,%fir_Considerado,%vendorlate,RPT_bajo,orders_with_fir_bajo,%_fir_bajo,fir_data_baja
1,828.0,Pita House,169,5.0,164.0,5.0,169.0,11.639053,2.115644,0.030488,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,5182.0,Il Romano,168,3.0,143.0,21.0,164.0,15.970414,1.988356,0.020979,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,5372.0,Jhot Pizza El Bosque,117,5.0,96.0,17.0,113.0,15.034188,2.115646,0.052083,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,5375.0,Jhot Pizza San Bernardo,273,16.0,251.0,19.0,270.0,14.754579,2.649471,0.063745,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,6212.0,Fuente Suiza Ñuñoa,1229,88.0,1180.0,43.0,1223.0,15.426829,2.457016,0.074576,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8834,484330.0,Papa John's Valle Grande,412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.516279,-11.946570,277.0,0.672330,0.068592,0.138462,1.383333,6.0,0.021661,1.0
8919,486489.0,Pollo Stop Parque Arauco 62,110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.271722,-13.373958,96.0,0.872727,0.052083,0.174757,1.203030,33.0,0.343750,1.0
9708,505635.0,Taco Bell Plaza Sur,112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.267088,-14.998148,108.0,0.964286,0.000000,0.132653,NaN,NaN,0.000000,1.0
10290,519648.0,McDonald´s - Macul Las Torres,1596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.460523,-9.754463,1098.0,0.687970,0.111111,0.191822,0.834783,253.0,0.230419,1.0


In [24]:
cambios_totales["DAY-RANGE"]="MONDAY-SUNDAY"
cambios_totales["HOUR-RANGE"]="0-23"
cambios_totales["PREPARATION-BUFFER"]="2"
cambios_totales["PREPARATION-TIME"]=cambios_totales["new_ept"]
cambios_totales["STRATEGY"]="OPS_TEMPORARY"

In [25]:
cambios_totales

,dim_partner_partner_id,dim_partner_partner_name,peya_order_id,is_vendor_late_10,is_vendor_late_nn,is_rider_late_10,is_rider_late_nn,estimated_prep_time,at_vendor_time_cleaned,%vendor_late,...,%vendorlate,RPT_bajo,orders_with_fir_bajo,%_fir_bajo,fir_data_baja,DAY-RANGE,HOUR-RANGE,PREPARATION-BUFFER,PREPARATION-TIME,STRATEGY
1,828.0,Pita House,169,5.0,164.0,5.0,169.0,11.639053,2.115644,0.030488,...,NaN,NaN,NaN,NaN,NaN,MONDAY-SUNDAY,0-23,2,8.0,OPS_TEMPORARY
12,5182.0,Il Romano,168,3.0,143.0,21.0,164.0,15.970414,1.988356,0.020979,...,NaN,NaN,NaN,NaN,NaN,MONDAY-SUNDAY,0-23,2,12.0,OPS_TEMPORARY
14,5372.0,Jhot Pizza El Bosque,117,5.0,96.0,17.0,113.0,15.034188,2.115646,0.052083,...,NaN,NaN,NaN,NaN,NaN,MONDAY-SUNDAY,0-23,2,11.0,OPS_TEMPORARY
17,5375.0,Jhot Pizza San Bernardo,273,16.0,251.0,19.0,270.0,14.754579,2.649471,0.063745,...,NaN,NaN,NaN,NaN,NaN,MONDAY-SUNDAY,0-23,2,11.0,OPS_TEMPORARY
23,6212.0,Fuente Suiza Ñuñoa,1229,88.0,1180.0,43.0,1223.0,15.426829,2.457016,0.074576,...,NaN,NaN,NaN,NaN,NaN,MONDAY-SUNDAY,0-23,2,12.0,OPS_TEMPORARY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8834,484330.0,Papa John's Valle Grande,412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.138462,1.383333,6.0,0.021661,1.0,MONDAY-SUNDAY,0-23,2,14.0,OPS_TEMPORARY
8919,486489.0,Pollo Stop Parque Arauco 62,110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.174757,1.203030,33.0,0.343750,1.0,MONDAY-SUNDAY,0-23,2,6.0,OPS_TEMPORARY
9708,505635.0,Taco Bell Plaza Sur,112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.132653,NaN,NaN,0.000000,1.0,MONDAY-SUNDAY,0-23,2,6.0,OPS_TEMPORARY
10290,519648.0,McDonald´s - Macul Las Torres,1596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.191822,0.834783,253.0,0.230419,1.0,MONDAY-SUNDAY,0-23,2,10.0,OPS_TEMPORARY
